# 115th U.S. Congress Composition and Voting Record - Data wrangling

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2">Libraries</a></span></li><li><span><a href="#Voting-record-data" data-toc-modified-id="Voting-record-data-3">Voting record data</a></span></li><li><span><a href="#Congress-members-data" data-toc-modified-id="Congress-members-data-4">Congress members data</a></span></li><li><span><a href="#Merge-voting-record-and-Congress-members-data" data-toc-modified-id="Merge-voting-record-and-Congress-members-data-5">Merge voting record and Congress members data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Senate-data" data-toc-modified-id="Senate-data-5.0.1">Senate data</a></span></li><li><span><a href="#House-of-representatives-data" data-toc-modified-id="House-of-representatives-data-5.0.2">House of representatives data</a></span></li></ul></li></ul></li><li><span><a href="#Current-Congress-members-demographic-data" data-toc-modified-id="Current-Congress-members-demographic-data-6">Current Congress members demographic data</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-7">Conclusion</a></span></li></ul></div>

## Introduction

**Background:** 

With the 2018 U.S. midterm elections right around the corner and the control of Congress at stake, the 115th U.S. Congress might soon drastically change. Indeed, the coming election features of the most __[diverse pool of candidates](https://www.nbcnews.com/politics/first-read/2018-candidate-diversity-goes-beyond-gender-n901956)__, from record numbers of women and LGBTQ candidates to unprecedented racial and religious diversity.

This project aims to analyze the composition and voting record of the 115th U.S. Congress on the eve of the 2018 midterm elections.

**Goal:**
<ul>
    <li>Prepare data for the <a href='congress-votes-analysis-kmeans.ipynb'>analysis phase</a>.</li>
</ul>

## Libraries

In [1]:
import numpy as np
import pandas as pd

## Voting record data

In [2]:
# Voting record dataset
votes_df = pd.read_csv('data/HS115_votes.csv')

# Preview of the dataset
votes_df.head()

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,115,House,1,10713,6,100.0
1,115,House,1,14066,1,98.8
2,115,House,1,14263,6,100.0
3,115,House,1,14657,1,98.7
4,115,House,1,14854,1,99.7


In [3]:
# Drop irrelevant columns from votes df
votes_cols = ['congress', 'prob']
votes_df = votes_df.drop(votes_cols, axis=1)

In [4]:
votes_df['cast_code'].value_counts().to_frame()

,cast_code
1,323250
6,195988
9,19752
7,388


In [5]:
# Change values for votes (1: yes vote, 5: no vote, 7 or 9: abstention)
votes_map = {1: 1, 6: 0, 7 : 0.5, 9: 0.5} 
votes_df['cast_code'] = votes_df['cast_code'].map(votes_map)

In [6]:
# Add a vote column
votes_map = {1: 'Yea', 0: 'Nay', 0.5 : 'Abstention'} 
votes_df['vote'] = votes_df['cast_code'].map(votes_map)

# Rename 'icpsr' column
votes_df = votes_df.rename(columns={'icpsr': 'member_id'})
votes_df.head()

,chamber,rollnumber,member_id,cast_code,vote
0,House,1,10713,0.0,Nay
1,House,1,14066,1.0,Yea
2,House,1,14263,0.0,Nay
3,House,1,14657,1.0,Yea
4,House,1,14854,1.0,Yea


## Congress members data

In [7]:
# Members
congress_df = pd.read_csv('data/HS115_members.csv')

# Preview dataset
congress_df.head()

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,115,President,99912,99,0,USA,200,0.0,0.0,"TRUMP, Donald John",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,115,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",...,NaN,0.338,0.446,-100.47503,0.89514,907.0,61.0,NaN,0.396,0.438
2,115,House,21102,41,7,AL,100,NaN,NaN,"SEWELL, Terri",...,NaN,-0.390,0.404,-88.42321,0.90293,866.0,33.0,NaN,-0.399,0.360
3,115,House,21192,41,2,AL,200,NaN,NaN,"ROBY, Martha",...,NaN,0.367,0.659,-62.38415,0.93325,903.0,22.0,NaN,0.407,0.546
4,115,House,21193,41,5,AL,200,NaN,NaN,"BROOKS, Mo",...,NaN,0.601,-0.429,-147.28461,0.84393,868.0,69.0,NaN,0.659,-0.544


In [8]:
# Drop irrelevant columns from members df
keep_cols =  ['chamber', 'icpsr', 'state_icpsr', 'state_abbrev', 'party_code', 'bioname']
congress_df = congress_df.filter(keep_cols)

In [9]:
congress_df['chamber'].value_counts().to_frame()

,chamber
House,445
Senate,105
President,1


In [10]:
# Remove members that are not part of Congress (i.e. President)
congress_df = congress_df[congress_df['chamber'] != 'President']

In [11]:
congress_df['party_code'].value_counts().to_frame()

,party_code
200,304
100,244
328,2


In [12]:
# Add a 'party' column
party_map = {100: 'D', 200: 'R', 328: 'I'} # Democrat, republican, independent
congress_df['party'] = congress_df['party_code'].map(party_map)
congress_df = congress_df.drop(['party_code'], axis=1)

In [13]:
# Split 'bioname' column into 'last_name' and 'first_name' columns
congress_df[['last_name', 'first_name']] = congress_df['bioname'].str.split(',', expand=True, n=1)
congress_df['last_name'] = congress_df['last_name'].str.capitalize()
congress_df = congress_df.drop('bioname', axis=1)

In [14]:
# Rename 'icpsr' and 'state_icpsr' columns
congress_df = congress_df.rename(columns={'icpsr': 'member_id', 'state_icpsr': 'state_id', 
                                          'state_abbrev': 'state_code'})

# Re-order columns
new_order = ['chamber', 'member_id', 'last_name', 'first_name', 'party', 'state_code', 'state_id']
congress_df = congress_df[new_order]
congress_df.head()

,chamber,member_id,last_name,first_name,party,state_code,state_id
1,House,20301,Rogers,Mike Dennis,R,AL,41
2,House,21102,Sewell,Terri,D,AL,41
3,House,21192,Roby,Martha,R,AL,41
4,House,21193,Brooks,Mo,R,AL,41
5,House,21376,Byrne,Bradley,R,AL,41


## Merge voting record and Congress members data

#### Senate data

In [15]:
# Create new dataframe for Senate votes only
senate_votes_df = votes_df[votes_df['chamber'] == 'Senate']

# Pivot Senate voting record dataframe based on Congress members' ID
senate_pivot_df = (senate_votes_df.pivot_table(index=['member_id'], columns='rollnumber', 
                                               values='cast_code').reset_index())

In [16]:
# Remove senators that were part of the 115th Congress for a very limited time (to avoid skewing the results)
minimum_votes = round(senate_votes_df['rollnumber'].nunique() * 0.25) # Must have voted for more than 25% of the bills

row_count = senate_pivot_df.shape[0]
senate_pivot_df = senate_pivot_df.dropna(thresh=minimum_votes)
new_row_count = senate_pivot_df.shape[0]
print(str(row_count - new_row_count) + " row(s) removed.")

senate_pivot_df = senate_pivot_df.fillna(0.5) # Fill missing values with abstentions

2 row(s) removed.


In [17]:
# Join voting record dataframe and Congress members dataframe
senate_df = pd.merge(congress_df, senate_pivot_df, on='member_id', how='inner')
senate_df.head()

,chamber,member_id,last_name,first_name,party,state_code,state_id,1,2,3,...,555,556,557,558,559,560,561,562,563,564
0,Senate,41704,Strange,"Luther Johnson, III",R,AL,41,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,Senate,41705,Jones,Gordon Douglas (Doug),D,AL,41,0.5,0.5,0.5,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2,Senate,94659,Shelby,Richard C.,R,AL,41,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Senate,40300,Murkowski,Lisa,R,AK,81,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Senate,41500,Sullivan,Daniel Scott,R,AK,81,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [18]:
# Export to CSV
senate_df.to_csv('data/clean_senate_votes.csv', index=False)

#### House of representatives data

In [19]:
# Create new dataframe for Senate votes only
house_votes_df = votes_df[votes_df['chamber'] == 'House']

# Pivot Senate voting record dataframe based on Congress members' ID
house_pivot_df = (house_votes_df.pivot_table(index=['member_id'], columns='rollnumber', values='cast_code')
                  .reset_index())

In [20]:
# Remove senators that were part of the 115th Congress for a very limited time (to avoid skewing the results)
row_count = house_pivot_df.shape[0]
house_pivot_df = house_pivot_df.dropna(thresh=minimum_votes)
new_row_count = house_pivot_df.shape[0]
print(str(row_count - new_row_count) + " row(s) removed.")

house_pivot_df = house_pivot_df.fillna(0.5) # Fill missing values with abstentions

8 row(s) removed.


In [21]:
# Joing voting record dataframe and Congress members dataframe
house_df = pd.merge(congress_df, house_pivot_df, on='member_id', how='inner')
house_df.head()

,chamber,member_id,last_name,first_name,party,state_code,state_id,1,2,3,...,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122
0,House,20301,Rogers,Mike Dennis,R,AL,41,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
1,House,21102,Sewell,Terri,D,AL,41,0.0,0.0,0.0,...,0.5,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,House,21192,Roby,Martha,R,AL,41,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
3,House,21193,Brooks,Mo,R,AL,41,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
4,House,21376,Byrne,Bradley,R,AL,41,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


In [22]:
# Export to CSV
house_df.to_csv('data/clean_house_votes.csv', index=False)

## Current Congress members demographic data

In [23]:
# Senators
senators_df = pd.read_csv("data/us-senate.csv")

# Preview of the dataset
senators_df.head()

,state_name,state_name_slug,state_code,state_code_slug,class,bioguide,thomas,govtrack,opensecrets,votesmart,...,address_city,address_state,address_zipcode,address_type,website,contact_page,facebook_url,twitter_handle,twitter_url,photo_url
0,Alaska,alaska,AK,ak,II,S001198,2290.0,412665,N00035774,114964,...,Washington,DC,20510,Building,http://www.sullivan.senate.gov,https://www.sullivan.senate.gov/contact/email,https://www.facebook.com/SenDanSullivan,SenDanSullivan,https://twitter.com/SenDanSullivan,https://cdn.civil.services/us-senate/headshots...
1,Alaska,alaska,AK,ak,III,M001153,1694.0,300075,N00026050,15841,...,Washington,DC,20510,Building,http://www.murkowski.senate.gov,https://www.murkowski.senate.gov/public/index....,https://www.facebook.com/SenLisaMurkowski,lisamurkowski,https://twitter.com/lisamurkowski,https://cdn.civil.services/us-senate/headshots...
2,Alabama,alabama,AL,al,II,J000300,NaN,412741,N00024817,176464,...,Washington,DC,20510,Building,https://www.jones.senate.gov/,https://www.jones.senate.gov/content/contact-s...,https://www.facebook.com/senatordougjones/,sendougjones,https://twitter.com/sendougjones,https://cdn.civil.services/us-senate/headshots...
3,Alabama,alabama,AL,al,III,S000320,1049.0,300089,N00009920,53266,...,Washington,DC,20510,Building,http://www.shelby.senate.gov,https://www.shelby.senate.gov/public/index.cfm...,https://www.facebook.com/RichardShelby,SenShelby,https://twitter.com/SenShelby,https://cdn.civil.services/us-senate/headshots...
4,Arkansas,arkansas,AR,ar,III,B001236,1687.0,400040,N00013873,27958,...,Washington,DC,20510,Building,http://www.boozman.senate.gov,https://www.boozman.senate.gov/public/index.cf...,https://www.facebook.com/JohnBoozman/,JohnBoozman,https://twitter.com/JohnBoozman,https://cdn.civil.services/us-senate/headshots...


In [24]:
# Representatives
representatives_df = pd.read_csv("data/us-house.csv")

# Preview of the dataset
representatives_df.head()

,state_name,state_name_slug,state_code,state_code_slug,district,at_large,vacant,bioguide,thomas,govtrack,...,address_city,address_state,address_zipcode,address_type,website,contact_page,facebook_url,twitter_handle,twitter_url,photo_url
0,Alaska,alaska,AK,ak,NaN,True,False,Y000033,1256.0,400440,...,Washington,DC,20515,NaN,https://donyoung.house.gov,http://donyoung.house.gov/contact/,https://www.facebook.com/RepDonYoung,repdonyoung,https://twitter.com/repdonyoung,https://cdn.civil.services/us-house/headshots/...
1,Alabama,alabama,AL,al,1.0,False,False,B001289,2197.0,412601,...,Washington,DC,20515,NaN,https://byrne.house.gov,https://byrne.house.gov/contact/email-me,https://www.facebook.com/RepByrne/,RepByrne,https://twitter.com/RepByrne,https://cdn.civil.services/us-house/headshots/...
2,Alabama,alabama,AL,al,2.0,False,False,R000591,1986.0,412394,...,Washington,DC,20515,NaN,https://roby.house.gov,https://roby.house.gov/contact-me,https://www.facebook.com/Representative.Martha...,RepMarthaRoby,https://twitter.com/RepMarthaRoby,https://cdn.civil.services/us-house/headshots/...
3,Alabama,alabama,AL,al,3.0,False,False,R000575,1704.0,400341,...,Washington,DC,20515,NaN,https://mikerogers.house.gov,https://mikerogers.house.gov/contact-mike/emai...,NaN,RepMikeRogersAL,https://twitter.com/RepMikeRogersAL,https://cdn.civil.services/us-house/headshots/...
4,Alabama,alabama,AL,al,4.0,False,False,A000055,1460.0,400004,...,Washington,DC,20515,NaN,https://aderholt.house.gov,https://aderholt.house.gov/contact-robert,https://www.facebook.com/RobertAderholt/,Robert_Aderholt,https://twitter.com/Robert_Aderholt,https://cdn.civil.services/us-house/headshots/...


In [25]:
# Correct spelling in 'LGBT' column name
senators_df = senators_df.rename(columns={'openley_lgbtq': 'openly_lgbtq'})

# Drop irrelevant columns
keep_cols = ['state_name', 'state_code', 'party', 'first_name', 'last_name', 'gender', 'ethnicity', 
             'religion', 'openly_lgbtq', 'date_of_birth']
senators_df = senators_df.filter(keep_cols)
representatives_df = representatives_df.filter(keep_cols)

# Add chamber columns:
senators_df['chamber'] = 'Senate'
representatives_df['chamber'] = 'House'

In [26]:
# Merge senators and representatives df
demo_df = pd.concat([senators_df, representatives_df], axis=0)
demo_df.head()

,state_name,state_code,party,first_name,last_name,gender,ethnicity,religion,openly_lgbtq,date_of_birth,chamber
0,Alaska,AK,republican,Daniel,Sullivan,male,white-american,roman-catholic,no,1964-11-13,Senate
1,Alaska,AK,republican,Lisa,Murkowski,female,white-american,roman-catholic,no,1957-05-22,Senate
2,Alabama,AL,democrat,Gordon,Jones,male,white-american,united-methodist,no,1954-05-04,Senate
3,Alabama,AL,republican,Richard,Shelby,male,white-american,presbyterian,no,1934-05-06,Senate
4,Arkansas,AR,republican,John,Boozman,male,white-american,southern-baptist,no,1950-12-10,Senate


In [27]:
# Rename 'ethnicity' column
demo_df = demo_df.rename(columns={'ethnicity': 'race'})


# Rename parties and some races
mapping = {'party': {'democrat': 'D', 'republican': 'R', 'independent': 'I'},
           'race': {'white-american': 'White', 'asian-american': 'Asian', 'hispanic-american': 'Hispanic',
                        'african-american': 'African-American', 'pacific-islander': 'Pacific Islander',
                        'native-american': 'Native American'},
           'openly_lgbtq': {'no': 'Straight'}
          }
demo_df.replace(mapping, inplace=True)

In [28]:
# Check for missing values
demo_df.columns[demo_df.isna().any()]

Index(['religion'], dtype='object')

In [29]:
# Fill in missing values in 'religion' column
demo_df['religion'].fillna('Unspecified', inplace=True)

# Update religion columns with broad religions instead of branches
religion_list = demo_df['religion'].unique().tolist()
religion_map = {religion: 'Christianity' for religion in religion_list}
non_christian = {'jewish': 'Judaism', 'muslim': 'Islam', 'hindu': 'Hinduism', 'buddhism': 'Buddhism',
                 'soka-gakkai-buddhist': 'Buddhism', 'jodo-shinshu-buddhist': 'Buddhism', 'deist': 'Deism',
                 'unspecified': 'Unspecified'}
religion_map.update(non_christian)
demo_df['religion'] = demo_df['religion'].map(religion_map)

In [30]:
# Change date of birth to age
demo_df['age'] = pd.to_numeric(demo_df['date_of_birth'].str.split('-').str[0]).apply(lambda x: 2018-x)
demo_df = demo_df.drop(['date_of_birth'], axis=1)

In [31]:
# Capitalize values
cap_cols = ['gender', 'religion', 'openly_lgbtq']
demo_df[cap_cols] = demo_df[cap_cols].apply(lambda x: x.str.capitalize())
demo_df.head()

,state_name,state_code,party,first_name,last_name,gender,race,religion,openly_lgbtq,chamber,age
0,Alaska,AK,R,Daniel,Sullivan,Male,White,Christianity,Straight,Senate,54
1,Alaska,AK,R,Lisa,Murkowski,Female,White,Christianity,Straight,Senate,61
2,Alabama,AL,D,Gordon,Jones,Male,White,Christianity,Straight,Senate,64
3,Alabama,AL,R,Richard,Shelby,Male,White,Christianity,Straight,Senate,84
4,Arkansas,AR,R,John,Boozman,Male,White,Christianity,Straight,Senate,68


In [32]:
# Export to CSV
demo_df.to_csv('data/clean_congress_demo.csv', index=False)

## Conclusion

The full analysis can be found in __[data analysis notebook](congress-votes-analysis-kmeans.ipynb)__.